In [ ]:
import os
import numpy as np
import pandas as pd
import scipy
import scipy.io
from scipy.io import loadmat
import matplotlib.pyplot as plt
import seaborn as sns
import shutil
import math
import ntpath
import sys
import logging
import time
import random

from importlib import reload
import plotly.graph_objects as go

import tensorflow as tf
from tensorflow import keras

In [ ]:
path = "../input/battery-data-set"
dis = os.listdir(path)
dis_mat = []
battery_grp = {
    
}
for i in dis :
    filtered_list = list(filter(lambda x:x.split('.')[-1] == 'mat',os.listdir(f"{path}/{i}")))
    battery_grp[i.split('BatteryAgingARC')[-1][1:]] = list(map(lambda x:x.split('.')[0],filtered_list))
    dis_mat.extend(list(map(lambda x:f"{path}/{i}/{x}",filtered_list)))

battery_grp['5_6_7_18'] = battery_grp['FY08Q4']
del battery_grp['FY08Q4']

dis_mat
battery_grp

In [ ]:
from scipy.io import loadmat

bs = [x.split('/')[-1].split('.')[0] for x in dis_mat]
print(bs)

ds = []
for b in dis_mat:
    ds.append(loadmat(b))

types = []
times = []
ambient_temperatures = []
datas = []

for i in range(len(ds)):
    x = ds[i][bs[i]]["cycle"][0][0][0]
#     ambient_temperatures.append(x['ambient_temperature'])
    ambient_temperatures.append(list(map(lambda y:y[0][0],x['ambient_temperature'])))
    types.append(x['type'])
    times.append(x['time'])
    datas.append(x['data']) 
    
batteries = []
cycles = []
for i in range(len(ds)):
    batteries.append(bs[i])
    cycles.append(datas[i].size)
    
battery_cycle_df = pd.DataFrame({'Battery':batteries,'Cycle':cycles}).sort_values('Battery',ascending=True)
battery_cycle_df.drop_duplicates(inplace=True)
battery_cycle_df

In [ ]:
Cycles = {}
params = ['Voltage_measured','Current_measured','Temperature_measured',
          'Current_load','Voltage_load','Time','Capacity',]

for i in range(len(bs)):
    Cycles[bs[i]] = {}
    Cycles[bs[i]]['count'] = 0 # This is true for battery B0005, 06, 07
#     ambient_temperatures[i]
    for param in params:
        Cycles[bs[i]][param] = []
        for j in range(datas[i].size):
            if types[i][j] == 'discharge':
                Cycles[bs[i]][param].append(datas[i][j][param][0][0][0])
        
    cap = []
    amb_temp = []
    for j in range(datas[i].size):
        if types[i][j] == 'discharge':
            cap.append(datas[i][j]['Capacity'][0][0][0])
            amb_temp.append(ambient_temperatures[i][j])
            
    Cycles[bs[i]]['Capacity'] = np.array(cap)
    Cycles[bs[i]]['ambient_temperatures'] = np.array(amb_temp)
Cycles = pd.DataFrame(Cycles)

battery_count = []
for b in battery_cycle_df['Battery'].values.tolist() :
    Cycles[b]['count'] = len(Cycles[b]['Capacity'])
Cycles

In [ ]:
Cycles['B0052']

In [ ]:
Cycles.loc['count',:].reset_index()
battery_cycle_df
compare_df = Cycles.loc['count',:].reset_index().set_index('index').join(battery_cycle_df.set_index('Battery')).rename(columns={'count':'After','Cycle':'Before'})
compare_df[['Before','After']]

In [ ]:
pd.DataFrame(Cycles['B0047'])

In [ ]:
df = pd.DataFrame({})
cols = ['Voltage_measured','Current_measured','Temperature_measured',
          'Current_load','Voltage_load','Time','Capacity','ambient_temperatures']
for col in cols :
    df[col] = Cycles['B0047'][col]
max_len = np.max(df['Time'].apply(lambda x:len(x)).values)
df_x = df.drop(columns=['Capacity']).values
df_y = df['Capacity'].values
df

In [ ]:
batteries

In [ ]:
df_all = pd.DataFrame({})
max_len = 0

experiment1 = ["B0005", "B0006", "B0007", "B0018"]
experiment2 = ["B0025", "B0026", "B0027", "B0028"]
experiment3 = ["B0029", "B0030", "B0031", "B0032"]
experiment4 = ["B0033", "B0034", "B0036"]
experiment5 = ["B0038", "B0039", "B0040"]
experiment6 = ["B0041", "B0042", "B0043", "B0044"]
experiment7 = ["B0045", "B0046", "B0047", "B0048"]
experiment8 = ["B0049", "B0050", "B0051", "B0052"]
experiment9 = ["B0053", "B0054", "B0055", "B0056"]

exp_try_out = experiment1 # replace karke wo experiment wala dalo
results_name = "critical_point_exp1_results"

for bat in exp_try_out:
    df = pd.DataFrame({})
    cols = ['Voltage_measured','Current_measured','Temperature_measured',
              'Current_load','Voltage_load','Time','Capacity','ambient_temperatures']
    for col in cols :
        df[col] = Cycles[bat][col]
    max_l = np.max(df['Time'].apply(lambda x:len(x)).values)
    max_len = max(max_l,max_len)
    df_all = pd.concat([df_all,df],ignore_index=True)
    
df=df_all.reset_index(drop=True)
df


flag = False
for i,j in enumerate(df['Capacity']) :
    try :
        if len(j) :
            df['Capacity'][i] = j[0] 
        else :
            df['Capacity'][i] = 0
        flag = True
    except :
        pass
print(flag)

## CRITICAL TIME POINTS FOR A CYCLE
## We will only these critical points for furthur training

## TEMPERATURE_MEASURED
## => Time at highest temperature

## VOLTAGE_MEASURED
## => Time at lowest Voltage

## VOLTAGE_LOAD
## => First time it drops below 1 volt after 1500 time


def getTemperatureMeasuredCritical(tm, time):
    high = 0
    critical = 0
    for i in range(len(tm)):
        if (tm[i] > high):
            high = tm[i]
            critical = time[i]
    return critical

def getVoltageMeasuredCritical(vm, time):
    low = 1e9
    critical = 0
    for i in range(len(vm)):
        if (vm[i] < low):
            low = vm[i]
            critical = time[i]
    return critical

def getVoltageLoadCritical(vl, time):
    for i in range(len(vl)):
        if (time[i] > 1500 and vl[i] < 1):
            return time[i]
    return -1


def fun(x) :
    cap = x['Capacity']
    amb_temp = x['ambient_temperatures']
    volt_load_c = getVoltageLoadCritical(x['Voltage_load'],x['Time'])
    volt_meas_c = getVoltageMeasuredCritical(x['Voltage_measured'],x['Time'])
    temp_meas_c = getTemperatureMeasuredCritical(x['Temperature_measured'],x['Time'])
    
    data = {
        'Capacity':cap,
        'ambient_temperatures':amb_temp,
        'Critical_Voltage_load':volt_load_c,
        'Critical_Voltage_measured':volt_meas_c,
        'Critical_Temperature_measured':temp_meas_c,
    }
    data_idx = [
        'Capacity', 'ambient_temperatures','Critical_Voltage_load',
        'Critical_Voltage_measured','Critical_Temperature_measured',
    ]
    y = pd.Series(data,index=data_idx)

    return y
df = df.apply(lambda x:fun(x),axis=1)
df

In [ ]:
from sklearn.metrics import max_error,mean_absolute_error,mean_squared_error,mean_squared_log_error
from sklearn.metrics import median_absolute_error,mean_absolute_percentage_error,r2_score

def get_scores(y_true,y_pred) :
    return {
        'max_error':max_error(y_true,y_pred),
        'mean_absolute_error':mean_absolute_error(y_true,y_pred),
        'mean_squared_error':mean_squared_error(y_true,y_pred),
        'mean_squared_log_error':mean_squared_log_error(y_true,y_pred),
        'median_absolute_error':median_absolute_error(y_true,y_pred),
        'mean_absolute_percentage_error':mean_absolute_percentage_error(y_true,y_pred),
        'r2_score':r2_score(y_true,y_pred)
    }

def get_preds(model,data_x) :
    return model.predict(data_x).clip(min=0)

# print(get_scores(train_y,get_preds(lr,train_x)))
# print(get_scores(test_y,get_preds(lr,test_x)))

df_x = df.drop(columns=['Capacity']).values
df_y = df['Capacity'].values

from sklearn.model_selection import train_test_split
train_x,test_x,train_y,test_y = train_test_split(df_x,df_y,test_size=0.2,random_state=0)
test_x,val_x,test_y,val_y = train_test_split(test_x,test_y,test_size=0.5,random_state=0)

train_x.shape,test_x.shape,val_x.shape,train_y.shape,test_y.shape,val_y.shape

In [ ]:
import pickle
from sklearn.linear_model import LinearRegression

lr = LinearRegression()
lr.fit(train_x,train_y)
model_name = "LR"
# print(get_scores(train_y,get_preds(lr,train_x)))
# print(get_scores(val_y,get_preds(lr,val_x)))
# print(get_scores(test_y,get_preds(lr,test_x)))

lr_results_train = pd.Series(get_scores(train_y,get_preds(lr,train_x)))
lr_results_val = pd.Series(get_scores(val_y,get_preds(lr,val_x)))
lr_results_test = pd.Series(get_scores(test_y,get_preds(lr,test_x)))
data = {"Train": lr_results_train,
        "Val": lr_results_val,
        "Test": lr_results_test}
lr_results = pd.DataFrame(data)
lr_results.to_csv(f"{model_name}_{results_name}.csv")
pickle.dump(lr, open(f"{model_name}_{results_name}", 'wb'))
lr_results 

In [ ]:
from sklearn.linear_model import LinearRegression,HuberRegressor
from sklearn.naive_bayes import BernoulliNB,CategoricalNB,ComplementNB,GaussianNB,MultinomialNB
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import LinearSVR,NuSVR,SVR
from sklearn.tree import DecisionTreeRegressor,ExtraTreeRegressor
from sklearn.ensemble import RandomForestRegressor,ExtraTreesRegressor,VotingRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor

algos = (LinearRegression,HuberRegressor,KNeighborsRegressor,
LinearSVR,NuSVR,
#          SVR, DecisionTreeRegressor,ExtraTreeRegressor,RandomForestRegressor,ExtraTreesRegressor,
XGBRegressor,LGBMRegressor,CatBoostRegressor)

params = {
    'silent':True
}

for algo in algos :
    model = algo()
    if type(model).__name__ == 'CatBoostRegressor' :
        model = algo(**params)
    print(type(model).__name__)
    model.fit(train_x,train_y)

    model_results_train = pd.Series(get_scores(train_y,get_preds(model,train_x)))
    model_results_val = pd.Series(get_scores(val_y,get_preds(model,val_x)))
    model_results_test = pd.Series(get_scores(test_y,get_preds(model,test_x)))
    data = {"Train": model_results_train,
            "Val": model_results_val,
            "Test": model_results_test}
    model_results = pd.DataFrame(data)
    print(model_results)
    print("~"*100)
    print("\n")
#     break

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split

from sklearn.metrics import max_error,mean_absolute_error,mean_squared_error,mean_squared_log_error
from sklearn.metrics import median_absolute_error,mean_absolute_percentage_error,r2_score

def get_scores(y_true,y_pred) :
    return np.sqrt(mean_squared_error(y_true,y_pred))

model_results = pd.DataFrame()

algos = (LinearRegression,HuberRegressor,KNeighborsRegressor,
LinearSVR,NuSVR,
#          SVR, DecisionTreeRegressor,ExtraTreeRegressor,RandomForestRegressor,ExtraTreesRegressor,
XGBRegressor,LGBMRegressor,CatBoostRegressor)

params = {
    'silent':True
}

for algo in algos :
    model = algo()
    if type(model).__name__ == 'CatBoostRegressor' :
        model = algo(**params)
    print(type(model).__name__)
    model.fit(train_x,train_y)

    model_results_train = (get_scores(train_y,get_preds(model,train_x)))
    model_results_val = (get_scores(val_y,get_preds(model,val_x)))
    model_results_test = (get_scores(test_y,get_preds(model,test_x)))
    data = {"Train": model_results_train,
            "Val": model_results_val,
            "Test": model_results_test}
    temp = pd.DataFrame(data,index=[f'Critical-Point_{type(model).__name__}'])
    model_results = model_results.append(temp)
model_results

In [ ]:
model_results.to_csv('critical_point_results.csv')